# Quick Selection

## 215. Kth Largest Element in an Array

A very intuitive thinking is to sort the array and the find the k-th largest, which is $O(N\log_N)$. How can we do better? We do not need to sort the whole array, but only the subarray that our target lies in. That's why we can utilize part of the solution in quick sort, that is the random partition algo and sovle the problem.

**Random Partition**: Randomly choose a number called `pivot`. And then move all the numbers smaller than it to the left, and all the numbers greater to its right. Return the idx of this random choose number.

- Swap the `pivot` to the rightmost number.
- Iterate all the remaining numbers, keep a pointer tracking the position that smaller number should be placed at. Any numbers to the left of this pointer are the smaller ones.
- Swap the `pivot` numbers from the right end back to the middle.

After the random partition, only this "lucky" random number is at its right place, but we also know the k-th largest number should be on its left or right subarray, which is why we can narrow the data.

We perform the same random partition on the subarray, and keep doing so, until the subarray converge to a single number, and we are able to find this k-th largest number.

1. Quick sort: https://leetcode.cn/problems/sort-an-array/solution/pai-xu-shu-zu-by-leetcode-solution/
2. k-th largest: https://leetcode.cn/problems/kth-largest-element-in-an-array/solution/partitionfen-er-zhi-zhi-you-xian-dui-lie-java-dai-/

In [43]:
class Solution:
    def findKthLargest(self, nums: list[int], k: int) -> int:
        def partition(nums, left, right):
            """returns the idx of a randomly choosen number after sorting.
            
            The array will be sorted in a way such that all the numbers to the left of this choosen number is smaller than it (in an unsorted way), and all the numbers to the right are larger ones.
            """
            import random
            pivot = random.randint(left, right)
            nums[pivot], nums[right] = nums[right], nums[pivot]
            idx = left
            for j in range(left, right):
                if nums[j] < nums[right]:
                    nums[j], nums[idx] = nums[idx], nums[j]
                    idx += 1
            nums[idx], nums[right] = nums[right], nums[idx]
            return idx
        
        n = len(nums)
        target = n - k
        left = 0
        right = n - 1
        while left <= right:
            idx = partition(nums, left, right)
            if idx == target:
                return nums[idx]
            elif idx < target:
                left = idx + 1
            else:
                right = idx - 1

## 238. Product of Array Except Self

Problem: Given an integer array nums, return an array answer such that `answer[i]` is equal to the product of all the elements of nums except `nums[i]`.

Solution: The problem requires us to solve within $O(n)$. The product except self can be viewed as products of its left and products of its right. Thus, we can loop the sequence twice, one to get all the left products, (intialze the `leftProd` of the first item to be 1). The second loop to get all the right products in the same way.

The the `res` should be straightforward.

In [3]:
class Solution:
    def productExceptSelf(self, nums: list[int]) -> list[int]:
        n = len(nums)
        res = [0] * n
        leftProd = [0] * n
        rightProd = [0] * n

        for i in range(n):
            if i == 0:
                leftProd[i] = 1
            else:
                leftProd[i] = leftProd[i-1] * nums[i-1]

        for i in range(n-1, -1, -1):
            if i == n - 1:
                rightProd[i] = 1
            else:
                rightProd[i] = rightProd[i + 1] * nums[i+1]
            res[i] = rightProd[i] * leftProd[i]

        return res


## 56. Merge Intervals

**Problem:** Given an array of intervals where `intervals[i] = [starti, endi]`, merge all overlapping intervals, and return an array of the non-overlapping intervals that cover all the intervals in the input.

In [18]:
class Solution:
    def merge(self, intervals: list[list[int]]) -> list[list[int]]:
        intervals.sort(key=lambda x: x[0])
        start, end = intervals[0]
        ans = list()
        for s, e in intervals[1:]:
            if s > end:
                ans.append([start, end])
                start, end =  s, e
            else:
                end = max(end, e)
        ans.append([start, end])
        return ans

In [19]:
Solution().merge([[1,3],[2,6],[8,10],[15,18]])

[[1, 6], [8, 10], [15, 18]]

# Prefix Sum

## 304. Range Sum Query 2D - Immutable

In [4]:
class NumMatrix:

    def __init__(self, matrix: list[list[int]]):
        m, n = len(matrix), len(matrix[0])
        self._sums = [[0] * (n + 1) for _ in range(m + 1)]

        for i in range(1, m + 1):
            for j in range(1, n + 1):
                self._sums[i][j] = self._sums[i][j-1] + self._sums[i - 1][j] - self._sums[i-1][j-1] + matrix[i-1][j-1]

    def sumRegion(self, row1: int, col1: int, row2: int, col2: int) -> int:
        _sums = self._sums
        return _sums[row2 + 1][col2 + 1] - _sums[row1][col2 + 1] - _sums[row2 + 1][col1] + _sums[row1][col1]


## 370. Range Addition

**Problem:** Given arrays like `updates = [start_idx, end_index, increase_amount`, return a sequence after all the updates, assume start with [0...0].

```
Input: length = 5, updates = [[1,3,2],[2,4,3],[0,2,-2]]
Output: [-2,0,3,5,3]

Input: length = 10, updates = [[2,4,6],[5,6,8],[1,9,-4]]
Output: [0,-4,2,2,2,4,4,-4,-4,-4]
```

**Solution:** Use a simple trick of prefix. Suppose we update from `l` to `r`, then do `d[l] += c` and `d[r+1] -= c`, then all the numbers after `r` will not be affected since the prefix sum offsets the value `c`. The previous numbers before `l` is free from impact also.

In [15]:
class Solution:
    def getModifiedArray(self, length: int, updates: list[list[int]]) -> list[int]:
        from itertools import accumulate
        delta = [0] * length
        for start, end, inc in updates:
            delta[start] += inc
            if end + 1 < length:
                delta[end + 1] -= inc
            print("The current delta:\t", delta)
            print("If accumulate now:\t", list(accumulate(delta)))
            print('-' * 20)
        return list(accumulate(delta))

In [16]:
Solution().getModifiedArray(5, [[1,3,2],[2,4,3],[0,2,-2]])

The current delta:	 [0, 2, 0, 0, -2]
If accumulate now:	 [0, 2, 2, 2, 0]
--------------------
The current delta:	 [0, 2, 3, 0, -2]
If accumulate now:	 [0, 2, 5, 5, 3]
--------------------
The current delta:	 [-2, 2, 3, 2, -2]
If accumulate now:	 [-2, 0, 3, 5, 3]
--------------------


[-2, 0, 3, 5, 3]

In [3]:
def count1(x):
    binary = format(x,'b')
    count = 0
    for i in binary:
        if i == '1': count += 1 
    return count


In [5]:
class Solution:
    def sortByBits(self, arr: list[int]) -> list[int]:
        n = list(arr)
        arr.sort()
        def count1(x):
            binary = format(x,'b')
            count = 0
            for i in binary:
                if i == '1': count += 1 
            return count
        arr.sort(key = lambda x: count1(x))
        return arr

In [6]:
Solution().sortByBits([1024,512,256,128,64,32,16,8,4,2,1])

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]